In [1]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt
import uuid

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base 
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from sqlalchemy import Column, Integer, String, Float


In [2]:
#Bring in the CSV files / data

csv_path = "Resources/covid_19_data.csv"
csv_path2 = "Resources/H1N1_2009.csv"

covid = pd.read_csv(csv_path, parse_dates=["ObservationDate"])
h1n1 = pd.read_csv(csv_path2, parse_dates=["Update Time"],encoding = 'unicode_escape')


In [3]:
covid
h1n1
covid

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,2020-01-22,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,2020-01-22,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,2020-01-22,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,2020-01-22,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
11294,11295,2020-04-02,Wyoming,US,2020-04-02 23:32:01,150.0,0.0,0.0
11295,11296,2020-04-02,Xinjiang,Mainland China,2020-04-02 23:32:01,76.0,3.0,73.0
11296,11297,2020-04-02,Yukon,Canada,2020-04-02 23:32:01,6.0,0.0,0.0
11297,11298,2020-04-02,Yunnan,Mainland China,2020-04-02 23:32:01,183.0,2.0,172.0


# COVID-19 Data Tables

In [4]:
covid = covid.loc[:,['ObservationDate', 'Province/State', 'Country/Region', 'Confirmed', 'Deaths', 'Recovered']]

#Rename Columns
covid = covid.rename(columns={"ObservationDate": "Date", "Province/State" : "Province", "Country/Region" : "Country"})

#Replace Values for country naming consistency
replace_values = {"(St. Martin)" : "St. Martin", "('St. Martin',)": "St. Martin", 
                  'Republic of Ireland' : "Ireland", 'Cabo Verde' : "Cape Verde" } 

covid = covid.replace({"Country": replace_values})

covid["Country"].unique()

array(['Mainland China', 'Hong Kong', 'Macau', 'Taiwan', 'US', 'Japan',
       'Thailand', 'South Korea', 'Singapore', 'Philippines', 'Malaysia',
       'Vietnam', 'Australia', 'Mexico', 'Brazil', 'Colombia', 'France',
       'Nepal', 'Canada', 'Cambodia', 'Sri Lanka', 'Ivory Coast',
       'Germany', 'Finland', 'United Arab Emirates', 'India', 'Italy',
       'UK', 'Russia', 'Sweden', 'Spain', 'Belgium', 'Others', 'Egypt',
       'Iran', 'Israel', 'Lebanon', 'Iraq', 'Oman', 'Afghanistan',
       'Bahrain', 'Kuwait', 'Austria', 'Algeria', 'Croatia',
       'Switzerland', 'Pakistan', 'Georgia', 'Greece', 'North Macedonia',
       'Norway', 'Romania', 'Denmark', 'Estonia', 'Netherlands',
       'San Marino', ' Azerbaijan', 'Belarus', 'Iceland', 'Lithuania',
       'New Zealand', 'Nigeria', 'North Ireland', 'Ireland', 'Luxembourg',
       'Monaco', 'Qatar', 'Ecuador', 'Azerbaijan', 'Czech Republic',
       'Armenia', 'Dominican Republic', 'Indonesia', 'Portugal',
       'Andorra', 'Latvia

In [5]:
covid = covid[['Country', 'Province', 'Date', 'Confirmed', 'Deaths', 'Recovered']]
covid

,Country,Province,Date,Confirmed,Deaths,Recovered
0,Mainland China,Anhui,2020-01-22,1.0,0.0,0.0
1,Mainland China,Beijing,2020-01-22,14.0,0.0,0.0
2,Mainland China,Chongqing,2020-01-22,6.0,0.0,0.0
3,Mainland China,Fujian,2020-01-22,1.0,0.0,0.0
4,Mainland China,Gansu,2020-01-22,0.0,0.0,0.0
...,...,...,...,...,...,...
11294,US,Wyoming,2020-04-02,150.0,0.0,0.0
11295,Mainland China,Xinjiang,2020-04-02,76.0,3.0,73.0
11296,Canada,Yukon,2020-04-02,6.0,0.0,0.0
11297,Mainland China,Yunnan,2020-04-02,183.0,2.0,172.0


In [6]:
#Group Provinces and take largest cumulative confirmed and death number
province_df = covid.groupby(by='Province').agg('max').reset_index(drop=False)

#Group all provinces into their countries and add confirmed and death numbers
province_df = province_df.groupby(by='Country').agg('sum').reset_index(drop=False)

province_df

,Country,Confirmed,Deaths,Recovered
0,Australia,5116.0,24.0,520.0
1,Canada,11308.0,138.0,14.0
2,Denmark,1513.0,13.0,84.0
3,France,34948.0,1433.0,2332.0
4,Germany,5.0,0.0,0.0
5,Hong Kong,802.0,4.0,147.0
6,Israel,8.0,0.0,0.0
7,Lebanon,2.0,0.0,0.0
8,Macau,41.0,0.0,10.0
9,Mainland China,81590.0,3318.0,76414.0


In [7]:
#Remove countries that are in province_df dataset
remove_list = province_df['Country']
global_covid_data = covid[~covid['Country'].isin(remove_list)]

#province_df
global_covid_data = global_covid_data.loc[:,['Country', 'Date', 'Confirmed', 'Deaths', 'Recovered']]
global_covid_data = global_covid_data.groupby(by='Country').agg('sum').reset_index(drop=False)


global_covid_data

,Country,Confirmed,Deaths,Recovered
0,Azerbaijan,1.0,0.0,0.0
1,Afghanistan,1735.0,39.0,41.0
2,Albania,2768.0,122.0,384.0
3,Algeria,6661.0,468.0,716.0
4,Andorra,3605.0,68.0,57.0
...,...,...,...,...
193,Vietnam,3381.0,0.0,956.0
194,West Bank and Gaza,912.0,8.0,142.0
195,Zambia,269.0,1.0,0.0
196,Zimbabwe,70.0,11.0,0.0


In [8]:
#Merge province and country data
global_covid_data = pd.concat([global_covid_data, province_df], ignore_index=True)

In [9]:
#global_covid_data consists of every country with affected persons - total number of; Confirmed, Deaths, Recovered

global_covid_data

,Country,Confirmed,Deaths,Recovered
0,Azerbaijan,1.0,0.0,0.0
1,Afghanistan,1735.0,39.0,41.0
2,Albania,2768.0,122.0,384.0
3,Algeria,6661.0,468.0,716.0
4,Andorra,3605.0,68.0,57.0
...,...,...,...,...
208,Netherlands,3722.0,138.0,12.0
209,Others,61.0,0.0,0.0
210,Taiwan,47.0,1.0,17.0
211,UK,5930.0,246.0,140.0


# H1N1 Data Tables

In [10]:
global_h1n1_data = h1n1.groupby(by='Country').agg('max').reset_index(drop=False)

#Rename Columns
global_h1n1_data = global_h1n1_data.rename(columns={"Cases": "Confirmed", "Update Time": "Date"})

global_h1n1_data


,Country,Confirmed,Deaths,Date
0,Algeria,5,0.0,2009-07-06 09:00:00
1,Antigua and Barbuda,2,0.0,2009-07-06 09:00:00
2,Argentina,2485,60.0,2009-07-06 09:00:00
3,Australia,5298,10.0,2009-07-06 09:00:00
4,Austria,19,0.0,2009-07-06 09:00:00
...,...,...,...,...
140,Venezuela,206,0.0,2009-07-06 09:00:00
141,Vietnam,181,0.0,2009-07-06 09:00:00
142,Virgin Islands,1,0.0,2009-07-06 09:00:00
143,West Bank and Gaza,60,0.0,2009-07-06 09:00:00


In [11]:
h1n1 = h1n1.rename(columns={"Cases": "Confirmed", "Update Time": "Date"})

h1n1

,Country,Confirmed,Deaths,Date
0,"Guernsey, Crown Dependency",5,0.0,2009-07-06 09:00:00
1,"Guernsey, Crown Dependency",5,0.0,2009-07-03 09:00:00
2,"Guernsey, Crown Dependency",5,0.0,2009-07-01 09:00:00
3,"Guernsey, Crown Dependency",1,0.0,2009-06-29 09:00:00
4,"Guernsey, Crown Dependency",1,0.0,2009-06-26 07:00:00
...,...,...,...,...
1817,Yemen,6,0.0,2009-06-26 07:00:00
1818,Yemen,6,0.0,2009-06-24 07:00:00
1819,Yemen,5,0.0,2009-06-22 07:00:00
1820,Yemen,4,0.0,2009-06-19 07:00:00


In [12]:
#Create lists of all countries
country_covid = global_covid_data['Country']
country_h1n1 = global_h1n1_data['Country']

#Combine country lists together
country_df = pd.concat([country_covid, country_h1n1], ignore_index=True)

#Put countries into a DataFrame
country_df = pd.DataFrame(country_df)

#Drop Duplicate Countries
country_df = country_df.drop_duplicates("Country")

#Reset Index and make new index as a column
country_df = country_df.reset_index(drop=True)
country_df = country_df.reset_index(level=0)

#Rename index column to Country ID
country_df = country_df.rename(columns={"index": "Country_ID"})

country_df = country_df[['Country', 'Country_ID']]

country_df



,Country,Country_ID
0,Azerbaijan,0
1,Afghanistan,1
2,Albania,2
3,Algeria,3
4,Andorra,4
...,...,...
234,"United Kingdom, Isle of Man, Crown Dependency",234
235,"United Kingdom, Jersey, Crown Dependency",235
236,Vanuatu,236
237,Virgin Islands,237


In [13]:
#Merge on global_covid_data
global_covid_data = pd.merge(global_covid_data, country_df, how='inner', on='Country')
global_covid_data = global_covid_data[['Country_ID', 'Country', 'Confirmed', 'Deaths', 'Recovered']]

In [14]:
#Merge on global_h1n1_data
global_h1n1_data = pd.merge(global_h1n1_data, country_df, how='inner', on='Country')
global_h1n1_data = global_h1n1_data[['Country_ID', 'Country', 'Confirmed', 'Deaths']]
global_h1n1_data

,Country_ID,Country,Confirmed,Deaths
0,3,Algeria,5,0.0
1,6,Antigua and Barbuda,2,0.0
2,7,Argentina,2485,60.0
3,198,Australia,5298,10.0
4,10,Austria,19,0.0
...,...,...,...,...
140,192,Venezuela,206,0.0
141,193,Vietnam,181,0.0
142,237,Virgin Islands,1,0.0
143,194,West Bank and Gaza,60,0.0


In [15]:
global_covid_data

,Country_ID,Country,Confirmed,Deaths,Recovered
0,0,Azerbaijan,1.0,0.0,0.0
1,1,Afghanistan,1735.0,39.0,41.0
2,2,Albania,2768.0,122.0,384.0
3,3,Algeria,6661.0,468.0,716.0
4,4,Andorra,3605.0,68.0,57.0
...,...,...,...,...,...
208,208,Netherlands,3722.0,138.0,12.0
209,209,Others,61.0,0.0,0.0
210,210,Taiwan,47.0,1.0,17.0
211,211,UK,5930.0,246.0,140.0


In [16]:
country_df.head(20)

,Country,Country_ID
0,Azerbaijan,0
1,Afghanistan,1
2,Albania,2
3,Algeria,3
4,Andorra,4
5,Angola,5
6,Antigua and Barbuda,6
7,Argentina,7
8,Armenia,8
9,Aruba,9


In [17]:
#Merge on covid
covid = pd.merge(covid, country_df, how='inner', on='Country')
covid = covid[['Country_ID', 'Country', 'Province', 'Date', 'Confirmed', 'Deaths', 'Recovered']]
covid

,Country_ID,Country,Province,Date,Confirmed,Deaths,Recovered
0,207,Mainland China,Anhui,2020-01-22,1.0,0.0,0.0
1,207,Mainland China,Beijing,2020-01-22,14.0,0.0,0.0
2,207,Mainland China,Chongqing,2020-01-22,6.0,0.0,0.0
3,207,Mainland China,Fujian,2020-01-22,1.0,0.0,0.0
4,207,Mainland China,Gansu,2020-01-22,0.0,0.0,0.0
...,...,...,...,...,...,...,...
11294,30,Burundi,NaN,2020-04-02,3.0,0.0,0.0
11295,162,Sierra Leone,NaN,2020-03-31,1.0,0.0,0.0
11296,162,Sierra Leone,NaN,2020-04-01,2.0,0.0,0.0
11297,162,Sierra Leone,NaN,2020-04-02,2.0,0.0,0.0


In [18]:
#Merge on h1n1
h1n1 = pd.merge(h1n1, country_df, how='inner', on='Country')
h1n1 = h1n1[['Country_ID', 'Country', 'Date', 'Confirmed', 'Deaths']]
h1n1

,Country_ID,Country,Date,Confirmed,Deaths
0,223,"Guernsey, Crown Dependency",2009-07-06 09:00:00,5,0.0
1,223,"Guernsey, Crown Dependency",2009-07-03 09:00:00,5,0.0
2,223,"Guernsey, Crown Dependency",2009-07-01 09:00:00,5,0.0
3,223,"Guernsey, Crown Dependency",2009-06-29 09:00:00,1,0.0
4,223,"Guernsey, Crown Dependency",2009-06-26 07:00:00,1,0.0
...,...,...,...,...,...
1817,238,Yemen,2009-06-26 07:00:00,6,0.0
1818,238,Yemen,2009-06-24 07:00:00,6,0.0
1819,238,Yemen,2009-06-22 07:00:00,5,0.0
1820,238,Yemen,2009-06-19 07:00:00,4,0.0


In [19]:
#Set Index for all dataframes
# country_df.set_index("Country ID", inplace=True)
# global_covid_data.set_index("Country ID", inplace=True)
# global_h1n1_data.set_index("Country ID", inplace=True)
# covid.set_index("Country ID", inplace=True)
# h1n1.set_index("Country ID", inplace=True)


In [20]:
global_covid_data

,Country_ID,Country,Confirmed,Deaths,Recovered
0,0,Azerbaijan,1.0,0.0,0.0
1,1,Afghanistan,1735.0,39.0,41.0
2,2,Albania,2768.0,122.0,384.0
3,3,Algeria,6661.0,468.0,716.0
4,4,Andorra,3605.0,68.0,57.0
...,...,...,...,...,...
208,208,Netherlands,3722.0,138.0,12.0
209,209,Others,61.0,0.0,0.0
210,210,Taiwan,47.0,1.0,17.0
211,211,UK,5930.0,246.0,140.0


In [21]:
global_h1n1_data

,Country_ID,Country,Confirmed,Deaths
0,3,Algeria,5,0.0
1,6,Antigua and Barbuda,2,0.0
2,7,Argentina,2485,60.0
3,198,Australia,5298,10.0
4,10,Austria,19,0.0
...,...,...,...,...
140,192,Venezuela,206,0.0
141,193,Vietnam,181,0.0
142,237,Virgin Islands,1,0.0
143,194,West Bank and Gaza,60,0.0


In [22]:
covid

,Country_ID,Country,Province,Date,Confirmed,Deaths,Recovered
0,207,Mainland China,Anhui,2020-01-22,1.0,0.0,0.0
1,207,Mainland China,Beijing,2020-01-22,14.0,0.0,0.0
2,207,Mainland China,Chongqing,2020-01-22,6.0,0.0,0.0
3,207,Mainland China,Fujian,2020-01-22,1.0,0.0,0.0
4,207,Mainland China,Gansu,2020-01-22,0.0,0.0,0.0
...,...,...,...,...,...,...,...
11294,30,Burundi,NaN,2020-04-02,3.0,0.0,0.0
11295,162,Sierra Leone,NaN,2020-03-31,1.0,0.0,0.0
11296,162,Sierra Leone,NaN,2020-04-01,2.0,0.0,0.0
11297,162,Sierra Leone,NaN,2020-04-02,2.0,0.0,0.0


In [23]:
country_df[['Country_ID', 'Country']]

,Country_ID,Country
0,0,Azerbaijan
1,1,Afghanistan
2,2,Albania
3,3,Algeria
4,4,Andorra
...,...,...
234,234,"United Kingdom, Isle of Man, Crown Dependency"
235,235,"United Kingdom, Jersey, Crown Dependency"
236,236,Vanuatu
237,237,Virgin Islands


   # Load DataFrames into SQlite DB

In [24]:
engine = create_engine("sqlite:///COVID19_vs_H1N1.sqlite")
conn = engine.connect()

In [25]:
country_df.to_sql('country', con=engine, index=False, if_exists='append')

In [26]:
global_covid_data.to_sql('global_covid_data', con=engine, index=True, if_exists='append')
#engine.execute("SELECT * FROM global_covid_data").fetchall()

In [27]:
global_h1n1_data.to_sql('global_h1h1_data', con=engine, index=True, if_exists='append')

In [28]:
covid.to_sql('covid', con=engine, index=True, if_exists='append')

In [29]:
h1n1.to_sql(name='h1n1', con=engine, index=True, if_exists='append')

In [30]:
engine.table_names()

['country', 'covid', 'global_covid_data', 'global_h1h1_data', 'h1n1']

# Load DataFrames into database

In [31]:
#country_df.to_sql(name='country', con=engine, index=True, if_exists='replace')

In [32]:
#global_covid_data.to_sql(name='global_covid_data', con=engine, index=True, if_exists='replace')

In [33]:
#global_h1n1_data.to_sql(name='global_h1h1_data', con=engine, index=True, if_exists='replace')

In [34]:
#covid.to_sql(name='covid', con=engine, index=True, if_exists='replace')

In [35]:
#h1n1.to_sql(name='h1n1', con=engine, index=True, if_exists='replace')